In [ ]:
## 算法 keras 自定义loss  Matlab data
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import os
import tensorflow_probability as tfp
import scipy.io as scio

# DeepNN topology
layers=[2,20,20,20,20,20,20,3]
# layers=[2,50,50,20,1]
tf_optimizer = tf.keras.optimizers.Adam(
   learning_rate=0.001, beta_1=0.99, epsilon=1e-1)
# tf_optimizer = tf.keras.optimizers.Adam()

# sampling points
dataFile = "x1"  
data = scio.loadmat(dataFile)
x1 = data['x1']
dataFile = "x2"  
data = scio.loadmat(dataFile)
x2 = data['x2']
x1 =tf.reduce_sum(x1, axis=0)
x2 =tf.reduce_sum(x2, axis=0)
x1=tf.cast(x1, dtype=tf.float32)
x2=tf.cast(x2, dtype=tf.float32)
#print(x1)

class mymodel():
    def __init__(self,layers,optimizer,x1,x2):
        self.u_model=tf.keras.Sequential()
        self.u_model.add(tf.keras.layers.InputLayer(input_shape=(layers[0],)))
        self.u_model.add(tf.keras.layers.Lambda(lambda X: X))
        for width in layers[1:7]:
            self.u_model.add(tf.keras.layers.Dense(
            width, activation=tf.nn.tanh,
            kernel_initializer='glorot_normal'))
        
        self.u_model.add(tf.keras.layers.Dense(
            layers[7],
            kernel_initializer='glorot_normal'))
        
        # Computing the sizes of weights/biases for future decomposition
        self.sizes_w = []
        self.sizes_b = []
        for i, width in enumerate(layers):
            if i != 1:
                self.sizes_w.append(int(width * layers[1]))
                self.sizes_b.append(int(width if i != 0 else layers[1]))
        
        self.optimizer=optimizer
        self.dtype=tf.float32

    # Defining custom loss
    def __loss(self):
        Xnode=tf.constant([4.636567,0.995896],shape=(1,2),dtype=tf.float32)
        u_pred=self.u_model(Xnode)
        u_pred0=u_pred[:,0]
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(x1)
            tape.watch(x2)
            X_f = tf.stack([x1,x2], axis=1)
            u = self.u_model(X_f)
            u0=u[:,0]
            u1=u[:,1]
            u2=u[:,2]
    
        u_x1=tape.gradient(u0, x1)
        #u_x1=tf.reduce_mean(u_x1,axis=1)
        u_x1=u_x1+2*x1-2*4.636567
        u_x2=tape.gradient(u0, x2)
        #u_x2=tf.reduce_mean(u_x2,axis=1)
        u_x2=u_x2+2*x2-2*0.995896
        
        rho=1.0
        K=10.0
        beta=3.0
        x0=1.0
        alpha=1.0
        lambda0=0.12
        R=1.55
        sigma1=0.1
        sigma2=1.0
        b1=rho*x1*(x2-x1/K)-beta*x1/(x1+x0)
        b2=R-alpha*x2-lambda0*x1*x2
        delta=0.001
        
        return tf.reduce_mean(tf.square(b1+0.5*sigma1**2*x1**4*u_x1-u1),axis=0) * 1 + \
            tf.reduce_mean(tf.square(b2+0.5*sigma2**2*u_x2-u2),axis=0) * 1 + \
            tf.reduce_mean((u_x1*u1+u_x2*u2)**2/((u_x1**2+u_x2**2)*(u1**2+u2**2)+delta),axis=0) * 1 + \
            tf.reduce_mean(tf.square(u_pred0),axis=0) * 0.1

    def __grad(self):
        with tf.GradientTape() as tape:
            loss_value = self.__loss()
        return loss_value, tape.gradient(loss_value, self.__wrap_training_variables())

    def __wrap_training_variables(self):
        var = self.u_model.trainable_variables
        return var

    def summary(self):
        return self.u_model.summary()

    # The training function
    def fit(self, tf_epochs=5000):
    # Creating the tensors
    #X_u = tf.convert_to_tensor(X_u, dtype=self.dtype)
    #u = tf.convert_to_tensor(u, dtype=self.dtype)
    #print(self.__wrap_training_variables())

        LOSS=np.zeros([1,tf_epochs])
        for epoch in range(tf_epochs):
            # Optimization step
            loss_value, grads = self.__grad()
            self.optimizer.apply_gradients(zip(grads, self.__wrap_training_variables()))
            LOSS[0,epoch]=loss_value.numpy()
            if np.mod(epoch,100)==0:
                print('epoch, loss_value:', epoch, loss_value)

        scio.savemat('LOSS.mat',{'LOSS':LOSS})

    def predict(self, X_star):
        u_star = self.u_model(X_star)
        # f_star = self.H_model()
        return u_star#, f_star


model=mymodel(layers, tf_optimizer, x1, x2)

# checkpoint_save_path="./checkpoint/mnist.ckpt"
# if os.path.exists(checkpoint_save_path + '.index'):
#   print('-------------------load the model---------------------')
#   model.load_weights(checkpoint_save_path)

# cp_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path,
#                         save_weights_only=True,
#                         save_best_only=True)

history=model.fit(tf_epochs=1000000)

model.summary()

# Getting the model predictions, from the same (x,t) that the predictions were previously gotten from
x11=tf.constant([4,5.5,7,1.6667,4.636567,7,1.5,4,7],dtype=tf.float32)
y11=tf.constant([0,0,0,1.2917,0.995896,1,2,2,2],dtype=tf.float32)
X_star = tf.stack([x11,y11], axis=1)
print('X_star:\n ', X_star)
u_pred= model.predict(X_star)
print('u_pred:\n ', u_pred)

dataFile = "xtest"  
data = scio.loadmat(dataFile)
xtest = data['xtest']
dataFile = "ytest"  
data = scio.loadmat(dataFile)
ytest = data['ytest']
xtest =tf.reduce_sum(xtest, axis=0)
ytest =tf.reduce_sum(ytest, axis=0)
X_star = tf.stack([xtest,ytest], axis=1)
Stest= model.predict(X_star)
scio.savemat('Stest.mat',{'Stest':Stest.numpy()})


In [ ]:
## computing MPEP and prefactor
rho=1.0
K=10.0
beta=3.0
x0=1.0
alpha=1.0
lambda0=0.12
R=1.55
sigma1=0.1
sigma2=1.0
detH_bar=0.1546
detH_star=0.2380
lambda_star=0.3721
pi=3.141592653

h=0.001
nT=10000
x1=np.zeros(nT,)
x2=np.zeros(nT,)
divL=np.zeros(nT,)
#normbx=np.zeros(nT,)
#x1[0]=1.6667+0.1
#x2[0]=1.2917
x1[0]=1.6667+0.1*0.8
x2[0]=1.2917-0.1*0.6

for epoch in range(nT-1):
    x01=tf.constant(x1[epoch],shape=(1,),dtype=tf.float32)
    x02=tf.constant(x2[epoch],shape=(1,),dtype=tf.float32)
    
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x01)
        tape.watch(x02)
        # Packing together the inputs
        X_f = tf.stack([x01,x02], axis=1)
        # Getting the prediction
        u = model.predict(X_f)
        u0=u[:,0]
        u1=u[:,1]
        u2=u[:,2]
    # Getting the derivative
    u_x = tape.gradient(u0, x01)
    u_y = tape.gradient(u0, x02)
    u_x = u_x+2*x01-2*4.636567
    u_y = u_y+2*x02-2*0.995896
    lx_x = tape.gradient(u1, x01)
    ly_y = tape.gradient(u2, x02)
    Ap = 4*sigma1**2*x01**3*u_x
    # Letting the tape go
    del tape
    divL[epoch]=lx_x.numpy()+ly_y.numpy()+Ap.numpy()
    
    if ((x1[epoch]-4.636567)**2+(x2[epoch]-0.995896)**2)<0.0025:
        break
    
    p1=u_x.numpy()
    p2=u_y.numpy()
    bx1=rho*x1[epoch]*(x2[epoch]-x1[epoch]/K)-beta*x1[epoch]/(x1[epoch]+x0)
    bx2=R-alpha*x2[epoch]-lambda0*x1[epoch]*x2[epoch]
    normb=(bx1**2+bx2**2)**0.5
    # normbx[epoch]=normb
    x1[epoch+1]=x1[epoch] -h * (bx1 + sigma1**2 *x1[epoch]**4 * p1)/normb
    x2[epoch+1]=x2[epoch] -h * (bx2 + sigma2**2 * p2)/normb

print('epoch:',epoch)
MPEP=tf.stack([x1[0:epoch+1],x2[0:epoch+1]], axis=1)
scio.savemat('MPEP.mat',{'MPEP':MPEP.numpy()})
print(MPEP)
integ=np.sum(divL[0:epoch+1])*h
prefactor=pi/lambda_star*(detH_star/detH_bar)**0.5*np.exp(integ)
print('integ:',integ)
print('prefactor:',prefactor)

In [ ]:
# computing MFPT
Dinv=40
xnode1=1.6667
xnode2=1.2917
xnode1=tf.constant(xnode1,shape=(1,),dtype=tf.float32)
xnode2=tf.constant(xnode2,shape=(1,),dtype=tf.float32)
X_f = tf.stack([xnode1,xnode2], axis=1)
u = model.predict(X_f)
u0=u[:,0]
U0=u0+(xnode1-4.636567)**2+(xnode2-0.995896)**2
MFPT=prefactor*np.exp(U0*Dinv)
print('U0:',U0)
print('MFPT:',MFPT)

In [ ]:
## finding the location of minimal quasi-potential on the boundary x1=3
rho=1.0
K=10.0
beta=3.0
x0=1.0
alpha=1.0
lambda0=0.12
R=1.55
sigma1=0.1
sigma2=1.0

xm1=3.0
xm2=1.0
eta=0.01

for epoch in range(1000):
    x01=tf.constant(xm1,shape=(1,),dtype=tf.float32)
    x02=tf.constant(xm2,shape=(1,),dtype=tf.float32)
    
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x01)
        tape.watch(x02)
        # Packing together the inputs
        X_f = tf.stack([x01,x02], axis=1)
        # Getting the prediction
        u = model.predict(X_f)
        u0=u[:,0]
    # Getting the derivative
    u_x2 = tape.gradient(u0, x02)
    u_x2 = u_x2+2*(x02-0.995896)
    # Letting the tape go
    del tape
    
    p2=u_x2.numpy()
    xm2=xm2-eta*p2
    print(epoch,xm1,xm2)

In [ ]:
## computing MPEP and prefactor non-characteristic boundary
x01=tf.constant(xm1,shape=(1,),dtype=tf.float32)
x02=tf.constant(xm2,shape=(1,),dtype=tf.float32)
with tf.GradientTape(persistent=True) as tape:
    tape.watch(x01)
    tape.watch(x02)
    X_f = tf.stack([x01,x02], axis=1)
    # Getting the prediction
    u = model.predict(X_f)
    u0=u[:,0]
    u1=u[:,1]
    u2=u[:,2]
    u_x2 = tape.gradient(u0, x02)
# Getting the derivative
u_x1 = tape.gradient(u0, x01)
u_x1 = u_x1+2*(x01-4.636567)
u_x2x2 = tape.gradient(u_x2, x02)
u_x2x2 = u_x2x2+2

detH_bar=0.1546
deth2_star=sigma2**2*u_x2x2.numpy()
miu2_star=-(0.5*sigma1**2*xm1**4*u_x1.numpy()+u1.numpy())
pi=3.141592653
print('deth2_star:',deth2_star)
print('miu2_star:',miu2_star)

h=0.001
nT=10000
x1=np.zeros(nT,)
x2=np.zeros(nT,)
divL=np.zeros(nT,)
#normbx=np.zeros(nT,)
x1[0]=xm1
x2[0]=xm2

for epoch in range(nT-1):
    x01=tf.constant(x1[epoch],shape=(1,),dtype=tf.float32)
    x02=tf.constant(x2[epoch],shape=(1,),dtype=tf.float32)
    
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x01)
        tape.watch(x02)
        # Packing together the inputs
        X_f = tf.stack([x01,x02], axis=1)
        # Getting the prediction
        u = model.predict(X_f)
        u0=u[:,0]
        u1=u[:,1]
        u2=u[:,2]
    # Getting the derivative
    u_x = tape.gradient(u0, x01)
    u_y = tape.gradient(u0, x02)
    u_x = u_x+2*x01-2*4.636567
    u_y = u_y+2*x02-2*0.995896
    lx_x = tape.gradient(u1, x01)
    ly_y = tape.gradient(u2, x02)
    Ap = 4*sigma1**2*x01**3*u_x
    # Letting the tape go
    del tape
    divL[epoch]=lx_x.numpy()+ly_y.numpy()+Ap.numpy()
    
    if ((x1[epoch]-4.636567)**2+(x2[epoch]-0.995896)**2)<0.01:
        break
    
    p1=u_x.numpy()
    p2=u_y.numpy()
    bx1=rho*x1[epoch]*(x2[epoch]-x1[epoch]/K)-beta*x1[epoch]/(x1[epoch]+x0)
    bx2=R-alpha*x2[epoch]-lambda0*x1[epoch]*x2[epoch]
    normb=(bx1**2+bx2**2)**0.5
    # normbx[epoch]=normb
    x1[epoch+1]=x1[epoch] -h * (bx1 + sigma1**2 *x1[epoch]**4 * p1)/normb
    x2[epoch+1]=x2[epoch] -h * (bx2 + sigma2**2 * p2)/normb

print('epoch:',epoch)
MPEP0=tf.stack([x1[0:epoch+1],x2[0:epoch+1]], axis=1)
scio.savemat('MPEP0.mat',{'MPEP0':MPEP0.numpy()})
print(MPEP0)
integ=np.sum(divL[0:epoch+1])*h
prefactor0=1/miu2_star*(2*pi*deth2_star/detH_bar)**0.5*np.exp(integ)
print('integ:',integ)
print('prefactor0:',prefactor0)

In [ ]:
# computing MFPT
Dinv=40
xnode1=xm1
xnode2=xm2
xnode1=tf.constant(xnode1,shape=(1,),dtype=tf.float32)
xnode2=tf.constant(xnode2,shape=(1,),dtype=tf.float32)
X_f = tf.stack([xnode1,xnode2], axis=1)
u = model.predict(X_f)
u0=u[:,0]
U0=u0+(xnode1-4.636567)**2+(xnode2-0.995896)**2
MFPT=prefactor0*(1/Dinv)**0.5*np.exp(U0*Dinv)
print('U0:',U0)
print('MFPT:',MFPT)